In [3]:
from spch_module.facilities import get_comp_by_name, get_summ_collections
from spch_module.mode import Mode
from spch_module.weight import Border, Weight,DEFAULT_BORDER

import numpy as np
mode = Mode(283, [20,20.55], 8.)
comp = get_comp_by_name(
    [
        'ГПА-ц3-16С-45-1.7(ККМ)',
        'ГПА Ц3 16с76-1.7М'],
    [1,1])
stage1 = comp[-1]
stage1

# sums = [stage1.calc_stage_summary_out(mode,freq) for freq in np.linspace(f_min, f_max, 10)]
# get_summ_collections([mode]*20, sums, [border])

Stage(type_spch=ГПА16-76 1.7, w_cnt=1, lim=Limit(r_val=500.8, k_val=1.31, plot_std=0.692, t_avo=293, dp_avo=0.06), idx=1, _w_cnt_current=None)

In [23]:
border = Border([
    Weight(16000, 7000, 100, 'mght', .1),
])
from spch_module.spch import Spch
self_obj = comp[0].type_spch
from spch_module.limit import DEFAULT_LIMIT, Limit
def get_freq_by_mght(self:Spch, percent_x:float,mght:float,lim:Limit)->float:
    koef_raskh = self.koef_raskh_by_percent_x(percent_x)
    koef_nap = self.calc_k_nap(koef_raskh)
    kpd_val = self.calc_k_kpd(koef_raskh)
    # mght = dh_val * mas_rate / kpd / (10**3)
    mas_rate = mode.q_in[0] * (10**6) / 24 / 60 / 60 * lim.plot_std
    dh_cur = mght / mas_rate * kpd_val * (10**3)
    z_in = .9
    
    m_t =  (lim.k_val - 1) / (lim.k_val * kpd_val)
    comp_degree = (dh_cur * m_t / (z_in * lim.r_val * mode.t_in) + 1) ** (1 / m_t)    
    return comp_degree
print(get_freq_by_mght(self_obj, 1, 16000, DEFAULT_LIMIT))
print(get_freq_by_mght(self_obj, 100, 16000, DEFAULT_LIMIT))



1.7511044941849598
1.4073170798659616
